In [21]:
import pandas as pd
import numpy as np

In [54]:
penetration_data = pd.read_excel('SVODpenetration.xlsx')
revenue_data = pd.read_excel('SVODrevenue.xlsx',  thousands=',')

In [55]:
new_penetration_data = penetration_data.set_index('Country').loc[:,"2023"]
new_revenue_data = revenue_data.set_index('Country').loc[:,"2023"]

In [58]:
df = pd.concat([new_penetration_data, new_revenue_data], axis=1, join='inner', keys=['Penetration', 'Revenue'])
df['Penetration_Heuristic'] = 4/np.log(1+df['Penetration']) # Heuristic function for the percentage growth of penetration that we expect to see
df.sort_values(by = ['Revenue'], ascending=False)[:30]

,Penetration,Revenue,Penetration_Heuristic
Country,,,
United States,46.7,39250.0,1.034947
China,28.3,19360.0,1.184277
United Kingdom,41.9,3892.0,1.064149
Germany,41.1,3718.0,1.069505
Japan,37.0,2937.0,1.099630
Canada,43.1,2165.0,1.056396
France,35.3,2105.0,1.113642
India,5.6,1696.0,2.119689
Brazil,14.4,1451.0,1.462861


The function 5/np.log(1+df['Penetration']) is representative of the rate of growth we can expect the market to grow as it is easier for penetration to go up the lower it already is. It is scaled to be more realistic by a factor of 10.